In [1]:
import pandas as pd
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import re
import ast
players_df = pd.read_csv('players_df.csv')

# First scrape all team Ids and save them in a dict

In [2]:
def get_teams(league_url, years, league_id):
    '''Scrape team information for certain leagues and given years.'''
    df = pd.DataFrame()
    if type(years) == int:
        years = [years]
    for year in years:
        url = league_url + f"/plus/?saison_id={year}"

        # exception handling
        r = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        html = urlopen(r)
        bs = BeautifulSoup(html, 'html.parser')
        
        # Get the team names
        team_rows = bs.find('table', {'class': 'items'}).find_all('td', {"class":"hauptlink no-border-links"})
        
        teams = {}
        for row in team_rows:
            team_name = row.text.strip().split(' \\')[0]
            team_href = row.find('a')['href']
            team_id = team_href.split('/')[4]
            teams[team_name]={'href': team_href, 'id': team_id}
        # TODO maybe also add the market value of the team
        # turn into df
        teams_df = pd.DataFrame.from_dict(teams, orient="index").reset_index(drop=False, names="team_name")
        teams_df["year"] = year
        teams_df["league_id"] = league_id
        teams_df["top_flight"] = 1
        df = pd.concat([df, teams_df], axis=0)
    return df

In [3]:
leagues = {"/bundesliga/startseite/wettbewerb/L1":1, "/premier-league/startseite/wettbewerb/GB1":2,
                "/primera-division/startseite/wettbewerb/ES1":3, "/serie-a/startseite/wettbewerb/IT1":4,
                "/ligue-1/startseite/wettbewerb/FR1":5, "/bundesliga/startseite/wettbewerb/L2":6, "/premier-league/startseite/wettbewerb/GB2":7,
               "/primera-division/startseite/wettbewerb/ES2":8, "/serie-a/startseite/wettbewerb/IT2":9,
               "/ligue-1/startseite/wettbewerb/FR2":10, "/liga-portugal/startseite/wettbewerb/PO1": 11, "/super-lig/startseite/wettbewerb/TR1": 12,
               "/jupiler-pro-league/startseite/wettbewerb/BE1": 13, "/eredivisie/startseite/wettbewerb/NL1": 14,
               "/major-league-soccer/startseite/wettbewerb/MLS1":15, "/3-liga/startseite/wettbewerb/L3": 16,
               "/saudi-professional-league/startseite/wettbewerb/SA1": 17, "/chinese-super-league/startseite/wettbewerb/CSL": 18}

combined_df = pd.DataFrame()
for league_url in leagues.keys():
    print(f"{list(leagues.keys()).index(league_url)+1}/{len(list(leagues.keys()))} - {datetime.datetime.now()}")
    df = get_teams("https://www.transfermarkt.com" + league_url, [2023], leagues[league_url])
    combined_df = pd.concat([df, combined_df], axis=0)

print(combined_df.head())


1/18 - 2023-10-26 11:20:45.459489


2/18 - 2023-10-26 11:20:48.674821
3/18 - 2023-10-26 11:20:52.459101
4/18 - 2023-10-26 11:20:55.062401
5/18 - 2023-10-26 11:20:58.931969
6/18 - 2023-10-26 11:21:01.232290
7/18 - 2023-10-26 11:21:04.905989
8/18 - 2023-10-26 11:21:08.767055
9/18 - 2023-10-26 11:21:15.032903
10/18 - 2023-10-26 11:21:17.623924
11/18 - 2023-10-26 11:21:20.464191
12/18 - 2023-10-26 11:21:23.904366
13/18 - 2023-10-26 11:21:26.916920
14/18 - 2023-10-26 11:21:28.506105
15/18 - 2023-10-26 11:21:30.611168
16/18 - 2023-10-26 11:21:34.569183
17/18 - 2023-10-26 11:21:46.069616
18/18 - 2023-10-26 11:21:48.994497
           team_name                                               href  \
0      Shanghai Port  /shanghai-port/startseite/verein/27190/saison_...   
1   Shanghai Shenhua  /shanghai-shenhua/startseite/verein/3183/saiso...   
2  Wuhan Three Towns  /wuhan-three-towns/startseite/verein/70657/sai...   
3   Shandong Taishan  /shandong-taishan/startseite/verein/3182/saiso...   
4      Beijing Guoan  /beijing-sinobo-

In [4]:
teams_id_dict = {row["id"]:row["team_name"] for i, row in combined_df.iterrows()}
print(teams_id_dict)

{'27190': 'Shanghai Port', '3183': 'Shanghai Shenhua', '70657': 'Wuhan Three Towns', '3182': 'Shandong Taishan', '3176': 'Beijing Guoan', '17276': 'Zhejiang FC', '65791': 'Chengdu Rongcheng', '4768': 'Changchun Yatai', '3187': 'Tianjin Jinmen Tiger', '35272': 'Cangzhou Mighty Lions', '38929': 'Meizhou Hakka', '9982': 'Henan FC', '1320': 'Shenzhen FC', '33304': 'Dalian Professional', '8536': 'Qingdao Hainiu', '46487': 'Nantong Zhiyun', '1114': 'Al-Hilal SFC', '18544': 'Al-Nassr FC', '18487': 'Al-Ahli SFC', '8023': 'Al-Ittihad Club', '9840': 'Al-Shabab FC', '7732': 'Al-Ettifaq FC', '28844': 'Al-Taawoun FC', '50531': 'Al-Fayha FC', '40039': 'Abha Club', '27221': 'Al-Fateh SC', '50532': 'Damac FC', '31008': 'Al-Riyadh SC', '71665': 'Al-Okhdood Club', '40079': 'Al-Tai FC', '9131': 'Al-Hazem SC', '17209': 'Al-Wehda FC', '22381': 'Al-Raed SFC', '6070': 'Al-Khaleej FC', '17': 'Borussia Dortmund II', '245': 'SC Freiburg II', '129': 'SG Dynamo Dresden', '4795': 'FC Ingolstadt 04', '1': '1.FC Saa

## Now match the df to the teams

In [5]:
columns_with_list_type = ["transfer_years", "transfer_hrefs", "transfer_club_ids", "main_position", "other_positions", "nationality"]
players_df["club"] = players_df["current_club"].apply(ast.literal_eval)
# for col in columns_with_list_type:
#     players_df[col] = players_df[col].apply(ast.literal_eval)
# players_df["club"] = players_df["current_club"].apply(ast.literal_eval)


ValueError: malformed node or string: nan

In [12]:
print(players_df["club"])
my_dict = {"club":0} 
for col, v in my_dict.items():
    for i, row in players_df.iterrows():
        if row["club"] != []:
            if str(v) == "all":
                players_df.at[i, f"{col}_id"] = str(list(row[col][:len(row[col])]))
            else:
                players_df.at[i, f"{col}_id"] = row[col][0]
        else:
            players_df.at[i, f"{col}_id"] = None

0       [51772]
1       [51772]
2       [51772]
3       [51772]
4         [543]
         ...   
9862      [108]
9863      [108]
9864      [108]
9865      [108]
9866      [108]
Name: club, Length: 9867, dtype: object


C:\Users\555ka\AppData\Local\Temp\ipykernel_5348\3702702967.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '51772' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  players_df.at[i, f"{col}_id"] = row[col][0]


In [13]:
players_df["team"] = players_df["club_id"].apply(lambda x: teams_id_dict[x] if x in teams_id_dict.keys() else None)
print(players_df["team"])

0                 FC Cincinnati
1                 FC Cincinnati
2                 FC Cincinnati
3                 FC Cincinnati
4       Wolverhampton Wanderers
                 ...           
9862         SV Wehen Wiesbaden
9863         SV Wehen Wiesbaden
9864         SV Wehen Wiesbaden
9865         SV Wehen Wiesbaden
9866         SV Wehen Wiesbaden
Name: team, Length: 9867, dtype: object


In [8]:
print(type(players_df["nationality"][0]))
print(players_df['nationality'].apply(ast.literal_eval))
players_df["nationality"] = players_df['nationality'].apply(ast.literal_eval)
players_df['nationality_id'] = players_df['nationality'].apply(lambda x: x[0] if x != [] else None)
print(players_df['nationality_id'])

<class 'str'>
0          [United States]
1          [United States]
2          [United States]
3          [United States]
4               [Colombia]
               ...        
9862             [Croatia]
9863             [Croatia]
9864    [Germany, Croatia]
9865           [Australia]
9866        [Canada, Cuba]
Name: nationality, Length: 9867, dtype: object
0       United States
1       United States
2       United States
3       United States
4            Colombia
            ...      
9862          Croatia
9863          Croatia
9864          Germany
9865        Australia
9866           Canada
Name: nationality_id, Length: 9867, dtype: object


In [19]:
# Replace NaN values in the 'main_position' column with an empty list
players_df['main_position'].fillna("[]", inplace=True)
players_df["other_positions"].fillna("[]", inplace=True)

# Use ast.literal_eval to convert the string representations to actual lists
players_df['main_position'] = players_df['main_position'].apply(ast.literal_eval)
players_df["other_positions"] = players_df["other_positions"].apply(ast.literal_eval)

players_df["all_positions"] = players_df.apply(lambda row: list(set(row['main_position']) | set(row['other_positions'])), axis=1)

ValueError: malformed node or string: ['Goalkeeper']

In [21]:
all_positions = []

for i in players_df['all_positions']:
    for pos in i:
        if pos not in all_positions:
            all_positions.append(pos)

all_positions

['Goalkeeper',
 'Centre-Back',
 'Right-Back',
 'Right Midfield',
 'Left-Back',
 'Left Midfield',
 'Attacking Midfield',
 'Defensive Midfield',
 'Central Midfield',
 'Second Striker',
 'Centre-Forward',
 'Left Winger',
 'Right Winger',
 'Sweeper']

In [29]:
mapping_positions = {'Left Midfield':'LM', 'Central Midfield':'CM', 'Defensive Midfield':'CDM',
                    'Right Midfield':'RM','Attacking Midfield':'CAM', 'Goalkeeper':'GK',
                    'Left Winger':'LW', 'Right-Back':'RB', 'Centre-Forward':'ST',
                    'Second Striker':'CF', 'Centre-Back':'CB', 'Left-Back':'LB',
                    'Right Winger':'RW','Sweeper':'CB'}

players_df["fifa_positions"] = players_df["all_positions"].apply(lambda x: [mapping_positions[pos] for pos in x if pos is not None] if len(x)>0 else [])

players_df["fifa_positions_1"] = players_df["fifa_positions"].apply(lambda x: ', '.join(x))

In [27]:
all_positions = []
for i in players_df['fifa_positions']:
    for pos in i:
        if pos not in all_positions:
            all_positions.append(pos)

all_positions

['GK',
 'CB',
 'RB',
 'RM',
 'LB',
 'LM',
 'CAM',
 'CDM',
 'CM',
 'CF',
 'ST',
 'LW',
 'RW']

In [10]:
def transform_mv(mv):
    if pd.isna(mv):
        return None

    # Use regular expressions to extract the numeric part and the multiplier (k or m)
    match = re.match(r'€(\d+(?:\.\d+)?)([kKmM]?)', mv)
    if not match:
        return None

    value = float(match.group(1))
    multiplier = match.group(2).lower()
    
    if multiplier == 'k':
        value *= 1000
    elif multiplier == 'm':
        value *= 1000000

    return int(value)

# Assuming you have imported pandas as pd
players_df["current_mv_id"] = players_df["current_mv"].apply(transform_mv)

In [18]:
print(players_df.Number.unique())
players_df["new_number"] = players_df["Number"].apply(lambda x: int(-1) if (x == "None")|(x=="-") else int(x))
print(players_df.new_number.unique())

['18' '36' '1' '25' '15' '21' '4' '32' '3' '34' '13' '2' '28' '42' '8' '5'
 '93' '26' '37' '10' '7' '58' '31' '19' '9' '17' '29' '14' '30' '24' '22'
 '27' '12' '16' '20' '11' '41' '77' '35' '33' '74' '44' '6' '43' '56' '23'
 '87' '99' '50' '68' '95' '98' '88' '72' '47' '-' '55' '59' '67' '54' '91'
 '94' '92' '80' '61' '45' '38' '97' '39' '46' '82' '40' '75' '65' '48'
 '90' '81' '52' '83' '64' '49' '84' '73' '89' '70' '66' '60' '71' '96'
 '51' '79' '76' '57' '62' '78' '63' '69' '53' '85' '86']
[18 36  1 25 15 21  4 32  3 34 13  2 28 42  8  5 93 26 37 10  7 58 31 19
  9 17 29 14 30 24 22 27 12 16 20 11 41 77 35 33 74 44  6 43 56 23 87 99
 50 68 95 98 88 72 47 -1 55 59 67 54 91 94 92 80 61 45 38 97 39 46 82 40
 75 65 48 90 81 52 83 64 49 84 73 89 70 66 60 71 96 51 79 76 57 62 78 63
 69 53 85 86]


In [25]:
print(players_df.columns)
display(players_df.head(20))

Index(['player_id', 'player_href', 'player_name', 'Birthday', 'Number',
       'transfer_hrefs', 'transfer_years', 'current_club', 'transfer_club_ids',
       'current_mv', 'max_mv', 'main_position', 'nationality',
       'other_positions', 'club', 'main_position_id', 'nationality_id',
       'all_positions', 'current_mv_id', 'club_id', 'team', 'new_number',
       'fifa_positions'],
      dtype='object')


,player_id,player_href,player_name,Birthday,Number,transfer_hrefs,transfer_years,current_club,transfer_club_ids,current_mv,...,other_positions,club,main_position_id,nationality_id,all_positions,current_mv_id,club_id,team,new_number,fifa_positions
0,972454,/roman-celentano/profil/spieler/972454,Roman Celentano,2000-09-14,18,[/indiana-hoosiers-indiana-university-/transfe...,"[2021, 2019]",['51772'],"[9747, 36387]",€1.00m,...,[],[51772],[,United States,[Goalkeeper],1000000.0,51772,FC Cincinnati,18,[GK]
1,375564,/evan-louro/profil/spieler/375564,Evan Louro,1996-01-19,36,[/without-club/transfers/verein/515/saison_id/...,"[2022, 2021, 2019, 2019, 2018, 2018, 2017, 201...",['51772'],"[515, 23565, 623, 39815, 623, 39815, 623, 3981...",€200k,...,[],[51772],[,United States,[Goalkeeper],200000.0,51772,FC Cincinnati,36,[GK]
2,216686,/alec-kann/profil/spieler/216686,Alec Kann,1990-08-08,1,[/atlanta-united-fc/transfers/verein/51663/sai...,"[2021, 2019, 2018, 2017, 2017, 2016, 2016, 201...",['51772'],"[51663, 64328, 51663, 4115, 51663, 4284, 51604...",€150k,...,[],[51772],[,United States,[Goalkeeper],150000.0,51772,FC Cincinnati,1,[GK]
3,1001936,/paul-walters/profil/spieler/1001936,Paul Walters,2004-04-11,25,[/northwestern-wildcats-northwestern-universit...,"[2022, 2022]",['51772'],"[46114, 69991]",€25k,...,[],[51772],[,United States,[Goalkeeper],25000.0,51772,FC Cincinnati,25,[GK]
4,716276,/yerson-mosquera/profil/spieler/716276,Yerson Mosquera,2001-05-02,15,[/fc-cincinnati/transfers/verein/51772/saison_...,"[2023, 2022, 2021, 2019]",['543'],"[51772, 543, 8172, 24463]",€3.00m,...,[],[543],[,Colombia,[Centre-Back],3000000.0,543,Wolverhampton Wanderers,15,[CB]
5,245893,/matt-miazga/profil/spieler/245893,Matt Miazga,1995-07-19,21,[/chelsea-fc/transfers/verein/631/saison_id/20...,"[2022, 2021, 2021, 2020, 2020, 2019, 2019, 201...",['51772'],"[631, 1108, 631, 58, 631, 1032, 631, 1032, 631...",€1.80m,...,[],[51772],[,United States,[Centre-Back],1800000.0,51772,FC Cincinnati,21,[CB]
6,307890,/nick-hagglund/profil/spieler/307890,Nick Hagglund,1992-09-14,4,[/toronto-fc/transfers/verein/11141/saison_id/...,"[2018, 2016, 2016, 2013]",['51772'],"[11141, 46823, 11141, 46113]",€1.00m,...,[Right-Back],[51772],[,United States,"[Centre-Back, Right-Back]",1000000.0,51772,FC Cincinnati,4,"[CB, RB]"
7,636800,/ian-murphy/profil/spieler/636800,Ian Murphy,2000-01-16,32,[/duke-blue-devils-duke-university-/transfers/...,"[2021, 2018]",['51772'],"[9824, 66422]",€1.00m,...,[],[51772],[,United States,[Centre-Back],1000000.0,51772,FC Cincinnati,32,[CB]
8,1088716,/joey-akpunonu/profil/spieler/1088716,Joey Akpunonu,2001-12-21,3,[/bowling-green-falcons-bowling-green-state-un...,"[2022, 2022, 2021, 2021, 2020]",['51772'],"[15963, 72996, 15963, 72996, 15963]",€50k,...,[],[51772],[,United States,[Centre-Back],50000.0,51772,FC Cincinnati,3,[CB]
9,1144246,/london-aghedo/profil/spieler/1144246,London Aghedo,2000-05-31,34,[/air-force-falcons-us-air-force-academy-/tran...,[2022],['51772'],[70566],€25k,...,[],[51772],[,United States,[Centre-Back],25000.0,51772,FC Cincinnati,34,[CB]


In [30]:
important_columns = ["player_name", 'Birthday', 'new_number', 'current_mv_id', 'team', "nationality_id", "fifa_positions_1"]
players_df_preprocessed = players_df[important_columns]
players_df_preprocessed = players_df_preprocessed.rename({"nationality_id":"nationality", "current_mv_id":"current_mv", "new_number":"number", "fifa_positions": "positions"}, axis=1)
players_df_preprocessed.to_csv("players_df_preprocessed_1.csv", index=False)